In [ ]:
## 라이브러리 설치
!pip install matplotlib
!pip install seaborn
!pip install -q xgboost
!pip install graphviz
!pip install -q ipywidgets
!pip install -q geopandas
!pip install geopy
!pip install haversine
!pip install swifter

##### 추가된 모듈 설치 #########
!pip install -q pandarallel # 데이터프레임 병렬처리

In [1]:
## 라이브러리 불러오기 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from datetime import datetime
import os
import warnings
import ipywidgets as widgets
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ipywidgets import interact, interact_manual
from geopy import distance
import geopandas as gpd
import multiprocessing
from tqdm import trange,tqdm_notebook,tqdm
from pandarallel import pandarallel

warnings.filterwarnings("ignore",category=np.VisibleDeprecationWarning)
warnings.filterwarnings('ignore')

# 차트 한글 폰트 설정
# plt.rcParams['font.family'] = 'NanumGothicCoding'
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 차트 음수 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 차트 기본 사이즈 설정
plt.rcParams["figure.figsize"] = (12,9)

# 모든 컬럼 표시
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_row', 100)
pd.set_option('display.max_seq_items', None)

/usr/local/lib/python3.6/dist-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
mdf3 = pd.read_csv('./01_merge_nqi_data.csv')
mdf3.head(10)

,Unnamed: 0,date_x,hour,min,tool,nw_mode,lat_x,lon_x,dl_speed1,dl_pdcp_speed,ul_pdcp_speed,dl_rlc_speed,ul_rlc_speed,dl_mac_speed,ul_mac_speed,rsrq,rsrp,sinr,cqi,pci,si_gu,dong_x,gnb_ID,DU_Name,RRH_ID,CELL_ID,PCI,RU_TYPE,RANK_INDEX,HPW,VPW,ETILT,LAT,LON,dist,Unnamed: 0.1,Day,gnb_id,NE_ID,cellid,ErabAddAtt,scg_sum,UL_PRB,DL_PRB,ConnEstabAtt,Redirection,ul_bler,dl_bler,RssiOverPathAvg,dl_mcs,layer_num,dl_mcs_layer,date,scg_rate,red_rate
0,0,220415,12,28,KDM,NR NSA,37.02868,127.06871,666906.0,701.084,0.524,478.522,0.612,481.134,0.635,-10.52,-86.19,16.05,9.00,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
1,1,220415,12,28,KDM,NR NSA,37.02868,127.06871,675954.0,739.836,0.512,523.313,0.602,525.672,0.639,-10.48,-84.79,16.21,9.58,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
2,2,220415,12,28,KDM,NR NSA,37.02868,127.06871,730488.0,760.664,0.563,519.345,0.654,520.489,0.686,-10.44,-83.79,17.86,10.31,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
3,3,220415,12,28,KDM,NR NSA,37.02868,127.06871,722396.0,744.606,0.551,485.773,0.635,493.634,0.641,-10.39,-83.14,18.79,10.18,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
4,4,220415,12,28,KDM,NR NSA,37.02868,127.06871,790589.0,821.757,0.542,556.202,0.642,557.199,0.681,-10.39,-80.36,23.32,9.50,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
5,5,220415,12,28,KDM,NR NSA,37.02868,127.06871,869066.0,979.357,0.677,719.250,0.778,722.565,0.839,-10.37,-76.73,24.53,9.00,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
6,6,220415,12,28,KDM,NR NSA,37.02868,127.06871,833076.0,768.168,0.634,538.635,0.750,541.276,0.713,-10.37,-76.21,25.24,9.15,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
7,7,220415,12,28,KDM,NR NSA,37.02868,127.06871,700298.0,694.456,0.550,459.941,0.633,460.218,0.652,-10.34,-76.37,25.34,9.00,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
8,8,220415,12,28,KDM,NR NSA,37.02868,127.06871,647062.0,724.589,0.556,460.253,0.653,464.958,0.717,-10.40,-80.42,21.38,8.92,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31
9,9,220415,12,28,KDM,NR NSA,37.02868,127.06871,792930.0,824.604,0.623,557.951,0.727,558.677,0.746,-10.39,-82.12,17.05,8.64,201,평택시,지제동,82340.0,NS00240168,NRKG38621S,2000.0,201.0,32.0,4.0,50.0,24.0,0.0,37.0744,127.0569,5190.7,549172,04-15,82340,DU_404250031,2000.0,603.8,37.5,0.938,3.038,378.2,12.5,16.151,6.789,-91.856,6.055,2.487,14.58,220415,6.21,3.31


# 2. 지도학습 : 회귀 & 분류
2-1) 머신러닝(ML) : regression analysis(회귀분석)

In [3]:
## 각각 회귀모델 별 성능 비교 
from sklearn.linear_model import LinearRegression,  Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score, mean_absolute_percentage_error
from tqdm.notebook import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler,Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
    
def runExperiments(x_train, y_train, x_test, y_test):

    models = []
    models.append(('LinearRegression', LinearRegression()))
    models.append(('ExtraTrees',ExtraTreesRegressor(bootstrap=True,n_estimators=300,n_jobs=-1,random_state=123)))
    models.append(('RandomForestRegressor', RandomForestRegressor(bootstrap=True,n_estimators=300,n_jobs=-1,random_state=123)))
#     models.append(('XGBRegressor', XGBRegressor(n_estimators=100, learning_rate=0.1, gamma=0, subsample=0.75, colsample_bytree=0.8, max_depth=10)))
    models.append(('XGBRegressor', XGBRegressor(n_estimators=300, learning_rate=0.2, gamma=0, subsample=0.75, colsample_bytree=0.8, max_depth=10)))
    models.append(('XGBRegressor2', XGBRegressor(n_estimators=400, learning_rate=0.2, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=10)))
    models.append(('Ridge', Ridge(alpha=10)))
    models.append(('Lasso', Lasso(alpha=0.01)))
#     models.append(('KNeighborsRegressor', KNeighborsRegressor()))
    models.append(('KNeighborsRegressor', KNeighborsRegressor(n_neighbors=5, weights='distance')))  ##distance : 가중평균 
#     models.append(('KNeighborsRegressor2', KNeighborsRegressor(n_neighbors=7, weights='distance')))  ##distance : 가중평균 
    models.append(('LGBMRegressor',LGBMRegressor(max_depth=8,num_leaves=200,min_data_in_leaf=100)))
#     models.append(('LGBMRegressor2',LGBMRegressor(n_estimators=400, num_leaves=64, subsample = 0.6, max_depth=128)))
    models.append(('DecisionTreeRegressor', DecisionTreeRegressor()))

    nameList, maeList, mseList, rmseList, mapeList, r2List = [], [], [], [], [], []
    for name, model in tqdm_notebook(models):
 # tqdm_notebook : 진행바 
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        nameList.append(name) # 모델명
        maeList.append(np.around(mean_absolute_error(y_test, y_pred),2)) # MAE
        mseList.append(np.around(mean_squared_error(y_test, y_pred),2)) # MSE
        rmseList.append(np.around(mean_squared_error(y_test, y_pred)**0.5,2)) # RMSE
        mapeList.append(np.around(mean_absolute_percentage_error(y_test, y_pred),4)) # mape_list
        r2List.append(np.around(r2_score(y_test, y_pred),4)) # R2-Score

    return pd.DataFrame({'model':nameList, 'MAE':maeList, 'MSE':mseList, 'RMSE':rmseList, 
                         'MAPE':mapeList, 'R2-Score':r2List})

In [4]:
#Scaler 사용하여 데이터 분포도 변환
# scaler = MinMaxScaler()
scaler = RobustScaler()
# 피처 및 타겟 지정(기본)
dt = mdf3.copy()
cols = ['rsrp','rsrq','sinr','cqi','dl_mac_speed']
target = 'dl_mac_speed'
x = dt[cols].drop(target, axis=1)
x = scaler.fit_transform(x)
x = pd.DataFrame(x, columns=cols[:4])
y = dt[cols].loc[:, target]

#  학습용, 평가용 데이터 분리
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=123)

# 각각의 모델에 대한 성능평가
result = runExperiments(x_train, y_train, x_test, y_test)
result.sort_values(['R2-Score'], ascending=[False], inplace=True)
result

  0%|          | 0/10 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100


,model,MAE,MSE,RMSE,MAPE,R2-Score
1,ExtraTrees,62.00,13221.60,114.99,4.785365e+14,0.8204
2,RandomForestRegressor,62.52,13461.13,116.02,4.675913e+14,0.8172
7,KNeighborsRegressor,47.80,15094.44,122.86,3.373292e+14,0.7950
9,DecisionTreeRegressor,58.17,22332.67,149.44,3.662030e+14,0.6967
4,XGBRegressor2,115.04,25366.81,159.27,7.351839e+14,0.6555
3,XGBRegressor,123.75,28204.61,167.94,8.083168e+14,0.6170
8,LGBMRegressor,149.72,39478.30,198.69,1.241527e+15,0.4638
0,LinearRegression,166.74,47704.14,218.41,1.350267e+15,0.3521
5,Ridge,166.74,47704.14,218.41,1.350270e+15,0.3521
6,Lasso,166.74,47704.14,218.41,1.350266e+15,0.3521


In [5]:
#############################################################################################################
## 변수들의 중요도 가시화 하기
#############################################################################################################
def draw_feature_importances(std, fi_df):
    fig, ax = plt.subplots(figsize=(10,6))
    fi_df.plot.bar(yerr=std, ax=ax)
    ax.set_title("Feature importances", fontsize=12)
    ax.set_ylabel("Mean decrease in impurity", fontsize=12)
    fig.autofmt_xdate (rotation=45)
    fig.tight_layout()
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.show()

model = ExtraTreesRegressor(bootstrap=True,n_estimators=300,n_jobs=-1,random_state=123)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
feature_importances = model.feature_importances_
std = feature_importances
fi_df = pd.Series(feature_importances, index=cols[:4])
draw_feature_importances(std, fi_df)

NameError: name 'draw_feature_importances' is not defined

In [ ]:
scaler = RobustScaler()
# sclaer = Normalizer()
dt = mdf3.copy()
cols = ['rsrp','rsrq','sinr','cqi','dist','DL_PRB','UL_PRB','dl_bler','ul_bler', 'dl_mcs_layer','dl_mac_speed']
target = 'dl_mac_speed'
x = dt[cols].drop(target, axis=1)
x = scaler.fit_transform(x)
x = pd.DataFrame(x, columns=cols[:10])
y = dt[cols].loc[:, target]

#  학습용, 평가용 데이터 분리
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.1,random_state=123,shuffle=True)
result2 = runExperiments(x_train, y_train, x_test, y_test)
result2.sort_values(['R2-Score'], ascending=[False], inplace=True)
result2

In [ ]:
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
feature_importances = model.feature_importances_
std = feature_importances
fi_df = pd.Series(feature_importances, index=cols[:4])
draw_feature_importances(std, fi_df)

In [ ]:
sns.set_theme(style="whitegrid")
sns.catplot(x='model',y='R2-Score',data=result,kind="bar", palette="dark", alpha=.6, height=6)
plt.show()

In [ ]:
##extra trees  model 
ext_model = ExtraTreesRegressor(bootstrap=True,n_estimators=300,n_jobs=-1,random_state=123)
ext_model.fit(x_train.values, y_train.values)
ext_pred = ext_model.predict(x_test)

# # KNN
# knn_model = KNeighborsRegressor(n_neighbors=5, weights='distance')
# knn_model.fit(x_train.values,y_train.values)
# knn_pred = knn_model.predict(x_test)

# #xg boost model 
# xgb_model = XGBRegressor(n_estimators=400, learning_rate=0.2, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=10)
# xgb_model.fit(x_train.values, y_train.values)
# xgb_pred = xgb_model.predict(x_test)

In [ ]:
# 하이퍼 파라미터 튜닝 
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
scaler = StandardScaler()
# scaler = MinMaxScaler()
### parameter 들을 dictionary 형태로 설정
# 1건이여도 리스트 형태로 해야 합니다 ! []
param_grid={'max_depth':[10,12,14],
            'n_estimators':[200,300,400],
            'learning_rate':[0.2],
            'colsample_bytree':[1]}
model = XGBRegressor()
# model = XGBRegressor(n_estimators=200, learning_rate=0.1, gamma=0, subsample=0.75, colsample_bytree=0.8, max_depth=10)
#cv : 3번 교차검증을 한다는 의미 
gcv=GridSearchCV(model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=5)
gcv.fit(x_train.values,y_train.values)
print('final params', gcv.best_params_)   # 최적의 파라미터 값 출력
print('best score', gcv.best_score_)      # 최고의 점수

gcv_df = pd.DataFrame(gcv.cv_results_)
gcv_df[['params','mean_test_score','rank_test_score']]
# gcv_df


2-2) 머신러닝(ML) : Classfication analysis(분류 분석)

In [ ]:
# 분류모델 데이터셋 생성
cdf = mdf4.copy()

## dl_speed 수치형에서 분류형으로 전환 : dl_speed0
cdf['dl_speed0'] = 0
cdf.loc[cdf['dl_speed1'] < 200, 'dl_speed0'] = 1
cdf.loc[(cdf['dl_speed1'] >= 200) & (cdf['dl_speed1'] < 300), 'dl_speed0'] = 2
cdf.loc[(cdf['dl_speed1'] >= 300) & (cdf['dl_speed1'] < 400), 'dl_speed0'] = 3
cdf.loc[(cdf['dl_speed1'] >= 400) & (cdf['dl_speed1'] < 500), 'dl_speed0'] = 4
cdf.loc[(cdf['dl_speed1'] >= 500) & (cdf['dl_speed1'] < 600), 'dl_speed0'] = 5
cdf.loc[(cdf['dl_speed1'] >= 600) & (cdf['dl_speed1'] < 700), 'dl_speed0'] = 6
cdf.loc[(cdf['dl_speed1'] >= 700) & (cdf['dl_speed1'] < 800), 'dl_speed0'] = 7
cdf.loc[(cdf['dl_speed1'] >= 800) & (cdf['dl_speed1'] < 900), 'dl_speed0'] = 8
cdf.loc[(cdf['dl_speed1'] >= 900) & (cdf['dl_speed1'] < 1000), 'dl_speed0'] = 9
cdf.loc[(cdf['dl_speed1'] >= 1000) & (cdf['dl_speed1'] < 1100), 'dl_speed0'] = 10
cdf.loc[(cdf['dl_speed1'] >= 1100) & (cdf['dl_speed1'] < 1200), 'dl_speed0'] = 11
cdf.loc[cdf['dl_speed1'] >= 1400, 'dl_speed0'] = 6

In [ ]:
#############################################################################################################
## 각 분류모델 별 성능 비교 & 변수들의 중요도 가시화 하기
#############################################################################################################
# 라이브러리 불러오기
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import explained_variance_score
from sklearn.svm import SVC
import graphviz

# 각 분류모델 별 성능 비교
def runExperiments(x_train, y_train, x_test, y_test):
    random_seed = 100
    models = []
    models.append(('LogisticRegression', LogisticRegression()))
    models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state=156)))
    models.append(('RandomForestClassifier', RandomForestClassifier(n_estimators=100, max_features=None, max_depth=14, min_samples_leaf=1, 
                                                min_samples_split=3, criterion='gini', n_jobs=-1)))
    models.append(('XGBClassifier', XGBClassifier(n_estimators=55, learning_rate=0.1,max_depth=7,gamma=0, min_child_weight=1,
                       subsample=0.8, eval_metric='logloss',colsample_bytree=0.8,n_jobs=-1)))   
    models.append(('KNeighborsClassifier', KNeighborsClassifier(n_jobs=-1)))
    models.append(('LGBMClassifier', LGBMClassifier(n_estimators=400)))
    models.append(('AdaBoostClassifier', AdaBoostClassifier(n_estimators=100)))
    #models.append(('SVC', SVC(gamma='auto')))
    
    classifiers, accuracies, precisions, recalls, f1_scores= [],[],[],[],[]
        
    # tqdm_notebook : 진행바 
    for classifier, model in tqdm_notebook(models):
        model.fit(x_train, y_train) # 모델학습
        y_pred = model.predict(x_test) # 예측
        classifiers.append(classifier) #모델명
        accuracies.append(np.around(accuracy_score(y_test, y_pred).mean(), 2)) #정확도
        #precisions.append(np.around(precision_score(y_test, y_pred, pos_label='positive', average='micro').mean(), 2))  #정밀도
        precisions.append(np.around(precision_score(y_test, y_pred, average='macro').mean(), 2))  #정밀도
        recalls.append(np.around(recall_score(y_test, y_pred, average='macro').mean(), 2)) #신뢰도
        f1_scores.append(np.around(f1_score(y_test, y_pred, average='macro').mean(), 2)) #f1 스코어
    return pd.DataFrame({'model':classifiers, 'accuracy':accuracies, 'precision':precisions,
                         'recall':recalls, 'f1_score':f1_scores})

# 변수들의 중요도 가시화 하기
def draw_feature_importances(std, fi_df):
    fig, ax = plt.subplots(figsize=(10,6))
    fi_df.plot.bar(yerr=std, ax=ax)
    ax.set_title("Feature importances", fontsize=12)
    ax.set_ylabel("Mean decrease in impurity", fontsize=12)
    fig.autofmt_xdate (rotation=45)
    fig.tight_layout()
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.show()

In [ ]:
#############################################################################################################
## 1. 공통 Feature : 각 분류 모델 생성 및 성능 비교
#############################################################################################################

# Scaler 사용하여 데이터 분포도 변환 : scaler = MinMaxScaler()
scaler = RobustScaler()

# 피처 및 타겟 지정(기본)
dt = cdf
cols = ['rsrp','rsrq','sinr','cqi','dl_mac_speed']
target = 'dl_speed0'
x = dt[cols].drop(target, axis=1)
col_x = list(x)
x = scaler.fit_transform(x)
x = pd.DataFrame(x, columns=col_x)
y = dt[cols].loc[:, target]
#  학습용, 평가용 데이터 분리
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.1,random_state=123,shuffle=True)

# 모델 생성 및 평가
results = runExperiments(x_train, y_train, x_test, y_test)
results.sort_values(['f1_score'], ascending=[False], inplace=True)
results.reset_index(drop=True, inplace=True)
results

In [ ]:
## 공통 Feature : 변수들의 중요도 가시화 하기
model = LGBMClassifier(n_estimators=400)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

feature_importances = model.feature_importances_
std = feature_importances
fi_df = pd.Series(feature_importances, index=col_x)

draw_feature_importances(std, fi_df)

In [ ]:
#############################################################################################################
## 2. 대체 Feature 추가 : 각 분류 모델 생성 및 성능 비교
#############################################################################################################

# Scaler 사용하여 데이터 분포도 변환 : scaler = MinMaxScaler()
scaler = Normalizer()

# 피처 및 타겟 지정
dt = cdf
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX','UL_PRB', 'DL_PRB','ul_bler', 'dl_bler', 'dl_mcs_layer','scg_rate', 'red_rate','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX','UL_PRB', 'DL_PRB','ul_bler', 'dl_bler', 'dl_mcs_layer','dl_speed0']
cols = ['rsrp','rsrq','sinr','cqi','dist','DL_PRB', 'UL_PRB','dl_bler', 'ul_bler', 'dl_mcs_layer','dl_mac_speed']
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX', 'DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX', 'DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','RANK_INDEX','DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist','DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist', 'dl_speed0']
target = 'dl_mac_speed'
x = dt[cols].drop(target, axis=1)
col_x = list(x)
x = scaler.fit_transform(x)
x = pd.DataFrame(x, columns=col_x)
y = dt[cols].loc[:, target]

# 학습용, 평가용 데이터 분리
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=123)

# 모델 생성 및 평가
results = runExperiments(x_train, y_train, x_test, y_test)
results.sort_values(['f1_score'], ascending=[False], inplace=True)
# results.reset_index(drop=True, inplace=True)
results

In [ ]:
## 대체 Feature 추가 : 변수들의 중요도 가시화 하기
#model = XGBRegressor(n_estimators=200, learning_rate=0.1, gamma=0, subsample=0.75, colsample_bytree=0.8, max_depth=10)
#model = DecisionTreeClassifier(random_state=156)

c_model = LGBMClassifier(n_estimators=400)
c_model.fit(x_train, y_train)
y_pred = model.predict(x_test)
feature_importances = model.feature_importances_
std = feature_importances
fi_df = pd.Series(feature_importances, index=col_x)
draw_feature_importances(std, fi_df)

In [ ]:
#############################################################################################################
## 3. LGBM모델 하이퍼 파라미터 튜닝
#############################################################################################################
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# 평가지표 출력하는 함수 설정
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    F1 = f1_score(y_test, y_pred, average='macro')
    #AUC = roc_auc_score(y_test, pred_proba, multi_class='ovr', average='macro')
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    #print('AUC: {:.4f}'.format(AUC))

# GridSearchCV 최적 파라미터 찾기    
lgbm_model = LGBMClassifier(n_estimators=200) 

params = {'num_leaves':[32, 64, 128], 
          'max_depth':[16, 64, 128, 160],
          'min_child_samples' : [30, 60, 100], 
          'subsample':[0.6, 0.8, 1]
}

# cv는 3으로 지정
gridcv = GridSearchCV(lgbm_model, param_grid=params, cv=3)
gridcv.fit(x_train, y_train, 
           early_stopping_rounds=30,
           eval_metric='logloss', 
           eval_set=[(x_train, y_train),(x_test, y_test)])

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)
lgbm_f1_score = f1_score(y_test, y_pred, average='macro')
print('F1-Score: {:.4f}'.format(lgbm_f1_score))

In [ ]:
## 3. LGBM모델 하이퍼 파라미터 튜닝- 적용

# 평가지표 출력하는 함수 설정
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    F1 = f1_score(y_test, y_pred, average='macro')
    #AUC = roc_auc_score(y_test, pred_proba, multi_class='ovr', average='macro')
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    #print('AUC: {:.4f}'.format(AUC))

# Scaler 사용하여 데이터 분포도 변환 : scaler = MinMaxScaler()
scaler = StandardScaler()

# 피처 및 타겟 지정
dt = cdf
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX','UL_PRB', 'DL_PRB','ul_bler', 'dl_bler', 'dl_mcs_layer','scg_rate', 'red_rate','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX','UL_PRB', 'DL_PRB','ul_bler', 'dl_bler', 'dl_mcs_layer','dl_speed0']
cols = ['rsrp','rsrq','sinr','cqi','dist','UL_PRB', 'DL_PRB','ul_bler', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX', 'DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist','RANK_INDEX', 'DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','RANK_INDEX','DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist','DL_PRB', 'dl_bler', 'dl_mcs_layer','dl_speed0']
#cols = ['rsrp','rsrq','sinr','cqi','dist', 'dl_speed0']
target = 'dl_speed0'

x = dt[cols].drop(target, axis=1)
col_x = list(x)
x = scaler.fit_transform(x)
x = pd.DataFrame(x, columns=col_x)
y = dt[cols].loc[:, target]

# 학습용, 평가용 데이터 분리
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=123)

## 파라미터 적용
#lgbm_model = LGBMClassifier(n_estimators=400, num_leaves=64, subsample = 0.8, min_child_sample=60, max_depth=128)
lgbm_clf = LGBMClassifier(n_estimators=400, num_leaves=64, subsample = 0.6, min_child_sample=30, max_depth=128)
evals=[(x_train, y_train),(x_test, y_test)]
lgbm_clf.fit(x_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)
y_pred = lgbm_clf.predict(x_test)

get_clf_eval(y_test, y_pred)

In [ ]:
## plot_importance() 를 이용한 피처 중요도 시각화

from lightgbm import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(lgbm_clf, ax=ax)

In [ ]:
import joblib
# joblib.dump(knn_model, './220522/220522_knn.pkl')
joblib.dump(ext_model, './220522/220522_ext.pkl')
# joblib.dump(xgb_model, './220522/220522_xgb.pkl')

In [ ]:
#데이터, 모델 추출
#데이터 추출
# tdf2.to_csv('./220511_merge_statistics.csv')
# mdf3.to_csv('./merge_data_0504.csv')
sdf2.to_csv('./merge_site.csv')
import joblib
#xg boost 모델 추출 
joblib.dump(model, './xgbreg.pkl')
#lgbm  모델 추출
# joblib.dump(lgbm_clf, './lgbm_clf_v3.pkl')
#lgbm  모델 가져오기
c_model = joblib.load('./lgbm_clf_v3.pkl')
r_model = joblib.load('./xgbreg.pkl')